In [2]:
%matplotlib inline

import os
import multiprocess as mp
import scipy.stats as ss
import numpy as np
import pandas as pd
import h5py
import bioframe.dask
import cooltools
import cooler
from dask import dataframe as dd
import glob 
# import bbi

from itertools import combinations_with_replacement as comb
import pyarrow.parquet
import pyarrow as pa

In [14]:
glob.glob('/net/levsha/share/sergpolly/2019-02-19-uC-Nils-remap/results/pairs_library/*nodups*pairs.gz')

['/net/levsha/share/sergpolly/2019-02-19-uC-Nils-remap/results/pairs_library/U54-H1ESC4DN-FA-DSG-MNase-R1-T3__hg38.null.nodups.pairs.gz',
 '/net/levsha/share/sergpolly/2019-02-19-uC-Nils-remap/results/pairs_library/U54-HFFc64DN-FA-DSG-MNase-R1-T2__hg38.null.nodups.pairs.gz',
 '/net/levsha/share/sergpolly/2019-02-19-uC-Nils-remap/results/pairs_library/U54-HFFc64DN-FA-DSG-MNase-R1-T5__hg38.null.nodups.pairs.gz',
 '/net/levsha/share/sergpolly/2019-02-19-uC-Nils-remap/results/pairs_library/U54-H1ESC4DN-FA-DSG-MNase-R1-T1__hg38.null.nodups.pairs.gz',
 '/net/levsha/share/sergpolly/2019-02-19-uC-Nils-remap/results/pairs_library/U54-H1ESC4DN-FA-DSG-MNase-R2-T2__hg38.null.nodups.pairs.gz',
 '/net/levsha/share/sergpolly/2019-02-19-uC-Nils-remap/results/pairs_library/U54-HFFc64DN-FA-DSG-MNase-R1-T1__hg38.null.nodups.pairs.gz',
 '/net/levsha/share/sergpolly/2019-02-19-uC-Nils-remap/results/pairs_library/U54-HFFc64DN-FA-DSG-MNase-R1-T3__hg38.null.nodups.pairs.gz',
 '/net/levsha/share/sergpolly/2019

In [3]:
! ls /net/levsha/share/sergpolly/2019-02-19-uC-Nils-remap/results/pairs_library/

U54-H1ESC4DN-FA-DSG-MNase-R1-T1__hg38.null.dedup.stats
U54-H1ESC4DN-FA-DSG-MNase-R1-T1__hg38.null.dups.bam
U54-H1ESC4DN-FA-DSG-MNase-R1-T1__hg38.null.dups.pairs.gz
U54-H1ESC4DN-FA-DSG-MNase-R1-T1__hg38.null.nodups.bam
U54-H1ESC4DN-FA-DSG-MNase-R1-T1__hg38.null.nodups.pairs.gz
U54-H1ESC4DN-FA-DSG-MNase-R1-T1__hg38.null.nodups.pairs.gz.px2
U54-H1ESC4DN-FA-DSG-MNase-R1-T1__hg38.null.unmapped.bam
U54-H1ESC4DN-FA-DSG-MNase-R1-T1__hg38.null.unmapped.pairs.gz
U54-H1ESC4DN-FA-DSG-MNase-R1-T2__hg38.null.dedup.stats
U54-H1ESC4DN-FA-DSG-MNase-R1-T2__hg38.null.dups.bam
U54-H1ESC4DN-FA-DSG-MNase-R1-T2__hg38.null.dups.pairs.gz
U54-H1ESC4DN-FA-DSG-MNase-R1-T2__hg38.null.nodups.bam
U54-H1ESC4DN-FA-DSG-MNase-R1-T2__hg38.null.nodups.pairs.gz
U54-H1ESC4DN-FA-DSG-MNase-R1-T2__hg38.null.nodups.pairs.gz.px2
U54-H1ESC4DN-FA-DSG-MNase-R1-T2__hg38.null.unmapped.bam
U54-H1ESC4DN-FA-DSG-MNase-R1-T2__hg38.null.unmapped.pairs.gz
U54-H1ESC4DN-FA-DSG-MNase-R1-T3__hg38.null.dedup.stats
U54-H1ESC4DN-FA-DSG-MNase-R1-T3

In [17]:
hg38 = bioframe.fetch_chromsizes('hg38')
chromsizes = bioframe.fetch_chromsizes('hg38')
chromosomes = list(chromsizes.index)
files = []
for item in os.listdir('/net/levsha/share/sergpolly/2019-02-19-HiC-Nils-remap/results/pairs_library/'):
    if 'nodup' not in item:
        continue
    if 'bam' in item:
        continue
    if 'px2' in item:
        continue
    if 'hg38' not in item:
        continue
#     if 'DSG' in item:
#         continue
    files.append(item)
# files = [file for file in files if ('ESC' in file)]
files = np.sort(np.array(files))
files = files[-1:]
files

array(['U54-HFFc6-p22-FA-DpnII-20170327__hg38.null.nodups.pairs.gz'],
      dtype='<U60')

In [9]:
chromsizes

chr1     248956422
chr2     242193529
chr3     198295559
chr4     190214555
chr5     181538259
chr6     170805979
chr7     159345973
chr8     145138636
chr9     138394717
chr10    133797422
chr11    135086622
chr12    133275309
chr13    114364328
chr14    107043718
chr15    101991189
chr16     90338345
chr17     83257441
chr18     80373285
chr19     58617616
chr20     64444167
chr21     46709983
chr22     50818468
chrX     156040895
chrY      57227415
chrM         16569
Name: length, dtype: int64

In [18]:
pairs = []
for file in files:
#     try:
        path =  f'/net/levsha/share/sergpolly/2019-02-19-HiC-Nils-remap/results/pairs_library/{file}'
        print(path)
        ppx = bioframe.dask.read_pairix(path,
        chromsizes=hg38, 
        names=['read_id', 'chrom1', 'pos1', 'chrom2', 'pos2', 'strand1', 'strand2', 'pair_type', 'mapq1', 'mapq2'])
        pairs.append(ppx)
#     except:
#         print('Error')
    

/net/levsha/share/sergpolly/2019-02-19-HiC-Nils-remap/results/pairs_library/U54-HFFc6-p22-FA-DpnII-20170327__hg38.null.nodups.pairs.gz


In [19]:
chunksize = int(1e6)
for i, pair in enumerate(pairs):
    name = files[i]
    name = name[0:name.find('.null')]
    print(name)
    for j, (chrom1, chrom2) in enumerate(comb(list(hg38.index), 2)):
        print(chrom1, chrom2)
        try:
            df = pair[chrom1, chrom2]
        except KeyError:
            print("Didn't find ", chrom1, chrom2)
            continue

        table = pa.Table.from_pandas(df.compute(), preserve_index=False)
        if j == 0:
            writer = pa.parquet.ParquetWriter(
                f'/net/levsha/share/sameer/U54/microc/pairq/{name}.pairs.pq',
                table.schema,
                compression='snappy',
                use_dictionary=True,
                version='2.0')
            writer.write_table(table, row_group_size=chunksize)
        else:
            writer.write_table(table, row_group_size=chunksize)

    writer.close()

U54-HFFc6-p22-FA-DpnII-20170327__hg38
chr1 chr1


ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



Traceback (most recent call last):
  File "/home/sameer/miniconda3/lib/python3.6/site-packages/IPython/core/interactiveshell.py", line 2910, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "<ipython-input-19-8ddfcce382b4>", line 14, in <module>
    table = pa.Table.from_pandas(df.compute(), preserve_index=False)
  File "/home/sameer/miniconda3/lib/python3.6/site-packages/dask/base.py", line 143, in compute
    (result,) = compute(self, traverse=False, **kwargs)
  File "/home/sameer/miniconda3/lib/python3.6/site-packages/dask/base.py", line 392, in compute
    results = get(dsk, keys, **kwargs)
  File "/home/sameer/miniconda3/lib/python3.6/site-packages/dask/threaded.py", line 75, in get
    pack_exception=pack_exception, **kwargs)
  File "/home/sameer/miniconda3/lib/python3.6/site-packages/dask/local.py", line 512, in get_async
    key, res_info, failed = queue_get(queue)
  File "/home/sameer/miniconda3/lib/python3.6/site-packages/dask/local.py", line 151, in q

KeyboardInterrupt: 

In [3]:
f = pa.parquet.ParquetFile(f'/net/levsha/share/sameer/U54/microc/pairq/U54-H1ESC4DN-FA-DSG-MNase-R1-T4__hg38.pairs.pq')

In [ ]:
f.metadata

In [ ]:
files